<a href="https://colab.research.google.com/github/CarmenTheodoraCraciun/HairClasification/blob/main/HariTextureClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Enviroment

In [1]:
!rm -rf ./HairClasification
!git clone https://github.com/CarmenTheodoraCraciun/HairClasification.git
!mv ./HairClasification/originalData/Straight/ ./HairClasification/originalData/straight
!mv ./HairClasification/originalData/Wavy/ ./HairClasification/originalData/wavy

Cloning into 'HairClasification'...
remote: Enumerating objects: 7609, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 7609 (delta 6), reused 6 (delta 5), pack-reused 7598 (from 1)
Receiving objects: 100% (7609/7609), 447.94 MiB | 14.18 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Updating files: 100% (4110/4110), done.


In [2]:
import os
import cv2
import shutil
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, SGD
from numba import cuda

##Optimizing the training environment

* Enabling mixed precision for optimal performance on the GPU

In [3]:
mixed_precision.set_global_policy('mixed_float16')

##Cleaning up the TensorFlow session

* to avoid memory issues

In [4]:
tf.keras.backend.clear_session()
gc.collect()

0

# Data processing


In [5]:
def is_image(file_path):
    try:
        img = cv2.imread(file_path)
        return img is not None
    except:
        return False

def resize_image(img, size):
    """
    Resizes an image using bilinear interpolation.

    Args:
        img: Input image.
        size: Desired size of the output image (width, height).

    Returns:
        Resized image.
    """
    original_height, original_width, _ = img.shape
    new_width, new_height = size
    resized_img = np.zeros((new_height, new_width, 3), dtype=np.uint8)

    for i in range(new_width):
        for j in range(new_height):
            #i, j = pixel in the resized image
            # x, y = pixel in the original image
            x = i * (original_width - 1) / (new_width - 1)
            y = j * (original_height - 1) / (new_height - 1)

            # Neighborhood values
            x0 = int(np.floor(x))
            x1 = min(x0 + 1, original_width - 1)
            y0 = int(np.floor(y))
            y1 = min(y0 + 1, original_height - 1)

            # Extract the intensity values ​​of neighbors
            Ia = img[y0, x0] # stanga sus
            Ib = img[y0, x1] # drepata sus
            Ic = img[y1, x0] # stanga jos
            Id = img[y1, x1] # dreapta jos

            # Calculates the weight of each neighboring to the final value
            wa = (x1 - x) * (y1 - y)
            wb = (x - x0) * (y1 - y)
            wc = (x1 - x) * (y - y0)
            wd = (x - x0) * (y - y0)

            # The final value of the new pixel
            pixel = wa * Ia + wb * Ib + wc * Ic + wd * Id
            resized_img[j, i] = np.round(pixel).astype(int)

    return resized_img

def augment_image(img, size=(128, 128), shear_range=0.1, zoom_range=0.1, horizontal_flip=True):
    """
    Preprocesses and augments an image by applying resizing, shearing, zooming, and horizontal flipping.

    Args:
        img: Input image.
        size: Desired size of the output image (width, height).
        shear_range: Range for random shearing.
        zoom_range: Range for random zooming.
        horizontal_flip: Whether to randomly flip the image horizontally.

    Returns:
        Preprocessed and augmented image.
    """
    rows, cols, ch = img.shape
    shear_factor = np.random.uniform(-shear_range, shear_range)
    M_shear = np.array([[1, shear_factor, 0], [0, 1, 0]], dtype=np.float32)
    img = cv2.warpAffine(img, M_shear, (cols, rows))

    zoom_factor = np.random.uniform(1 - zoom_range, 1 + zoom_range)
    img = cv2.resize(img, None, fx=zoom_factor, fy=zoom_factor)

    if zoom_factor < 1: # zoom out
        new_height, new_width = img.shape[:2]
        # Number of pixels added to padding (black) and its addition
        pad_height = (rows - new_height) // 2
        pad_width = (cols - new_width) // 2
        img = cv2.copyMakeBorder(img, pad_height, pad_height,
            pad_width, pad_width, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    else: # zoom in
        # Position of the starting point for cutting
        start_x = (img.shape[1] - cols) // 2
        start_y = (img.shape[0] - rows) // 2
        img = img[start_y:start_y + rows, start_x:start_x + cols]

    if horizontal_flip and np.random.random() < 0.5:
        img = cv2.flip(img, 1)

    img = cv2.resize(img, size)

    return img

def preprocess_images(input_dir, output_dir, size=(128, 128), augment_prob=0.3):
    """
    Resizes images to the specified size and saves them to the output directory.
    Applies augmentations to a random subset of images.

    Args:
        input_dir: Directory containing the input images.
        output_dir: Directory to save the preprocessed images.
        size: Desired size of the output images (width, height).
        augment_prob: Probability of applying augmentation to each image.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    print("Start processing data.")
    for category in os.listdir(input_dir):
        category_dir = os.path.join(input_dir, category)
        output_category_dir = os.path.join(output_dir, category)
        if not os.path.exists(output_category_dir):
            os.makedirs(output_category_dir)

        num_images = 0
        if os.path.isdir(category_dir):
            for idx, img_name in enumerate(os.listdir(category_dir)):
                img_path = os.path.join(category_dir, img_name)

                if is_image(img_path):
                    img = cv2.imread(img_path)
                    if img is not None:
                        if np.random.rand() < augment_prob:
                            img = augment_image(img, size=size)
                        else:
                            img = resize_image(img, size)
                        new_img_name = f"{category}_{idx}.png"
                        cv2.imwrite(os.path.join(output_category_dir, new_img_name), img)
                        num_images += 1
                    else:
                        print(f"Failed to load image: {img_path}")
                else:
                    print(f"Not an image: {img_path}")

        print(f"Folder {output_category_dir} has {num_images} images.")

    print("The data are processed.")

In [6]:
if not os.path.exists('./HairClasification/processData'):
  preprocess_images('./HairClasification/originalData', './HairClasification/processData', size=(128, 128), augment_prob=0.3)

In [7]:
# import shutil

# shutil.make_archive('/content/HairClasification/processData', 'zip', '/content/HairClasification/processData')
# from google.colab import files

# files.download('/content/HairClasification/processData.zip')

##Loading data with labels

In [8]:
def load_images_and_labels(input_dir, size=(128, 128)):
    images = []
    labels = []
    for category in os.listdir(input_dir):
        category_dir = os.path.join(input_dir, category)
        if os.path.isdir(category_dir):
            for img_name in os.listdir(category_dir):
                img_path = os.path.join(category_dir, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.resize(img, size)
                    images.append(img)
                    labels.append(category)
    return np.array(images), np.array(labels)

In [9]:
input_dir = './HairClasification/processData'
images, labels = load_images_and_labels(input_dir)

##Data Augmentation (second) and Normalization

* on all the images, using predefine methods
* normalization: to contain only 0 and 1 values

In [10]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255  # Normalization
)

# Apply augmentation to all images in the dataset
augmented_images = []
augmented_labels = []

for img, label in zip(images, labels):
    img = img[np.newaxis, ...]  # Expand dimensions for datagen flow
    gen = datagen.flow(img, batch_size=1)
    # Generate a few augmented images per original image (e.g., 3 augmentations each)
    for _ in range(3):
        aug_img = next(gen)[0]
        augmented_images.append(aug_img)
        augmented_labels.append(label)


images_augmented = np.array(augmented_images)
labels_augmented = np.array(augmented_labels)

##One-hot Encoding

* One-hot encoding is a technique used to convert categorical data into a numeric format that a machine learning model can understand
  * e.g. [0,0,0,1,0] means that the hair is straight.


In [11]:
# Convert `labels_augmented` labels to integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_augmented)

labels_categorical = to_categorical(labels_encoded).astype('float32')
print(f"Labels converted to categorical format. Shape: {labels_categorical.shape}")
print(f"Example labels (first 5):\n{labels_categorical[:5]}")

Labels converted to categorical format. Shape: (6150, 5)
Example labels (first 5):
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


##Split data

In [12]:
X_train, X_val, y_train, y_val = train_test_split(
    images_augmented, labels_categorical, test_size=0.25, random_state=42
)

# The drive generator
train_generator = datagen.flow(X_train, y_train, batch_size=32, shuffle=True)

# Validation generator
validation_generator = datagen.flow(X_val, y_val, batch_size=32)
print(f"Train generator: {len(train_generator)} batches, Total images: {len(X_train)}")
print(f"Validation generator: {len(validation_generator)} batches, Total images: {len(X_val)}")

Train generator: 145 batches, Total images: 4612
Validation generator: 49 batches, Total images: 1538


##Transformation into TensorFlow datasets

In [13]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: iter(train_generator), # Create an iterator from the training generator
    output_signature=( # Define the data structure
        tf.TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32), # Images
        tf.TensorSpec(shape=(None, labels_categorical.shape[1]), dtype=tf.float32) # Labels
    )
).prefetch(buffer_size=tf.data.AUTOTUNE) # Optimize data preloading

validation_dataset = tf.data.Dataset.from_generator(
    lambda: iter(validation_generator),
    output_signature=(
        tf.TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, labels_categorical.shape[1]), dtype=tf.float32)
    )
).prefetch(buffer_size=tf.data.AUTOTUNE)

#Building the model

## Loading into the VGG16 pre-trained model

* VGG16 is a well-known and powerful convolutional neural network (CNN) model developed by the Visual Geometry Group (VGG) research team at the University of Oxford
* Its name, VGG16, derives from its 16-layer deep neural network structure.
* We use pre-trained VGG16 as a feature extractor

In [14]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

for layer in base_model.layers:
    layer.trainable = False # Freeze layers to preserve learned features

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


##Building on top of VGG16

In [15]:
# Initialize the model as a sequential stack of layers
model = Sequential()

# Add the pre-trained base model
# This serves as a feature extractor to reuse learned features from another task
model.add(base_model)

# Add a convolutional layer with 64 filters and a 3x3 kernel, ReLU activation
# Padding is set to 'same' to preserve spatial dimensions
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))

# Add a max pooling layer to downsample the feature maps
# This reduces spatial dimensions and captures the strongest signals in regions
model.add(MaxPooling2D((2, 2)))

# Add another convolutional layer with 128 filters and a 3x3 kernel, ReLU activation
# Padding is again set to 'same' to preserve dimensions
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

# Add another max pooling layer to further downsample feature maps
model.add(MaxPooling2D((2, 2)))

# Flatten the output feature maps into a 1D vector for the dense layers
model.add(Flatten())

# Add a dense (fully connected) layer with 256 neurons and ReLU activation
# Apply L2 regularization to discourage large weights and avoid overfitting
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))

# Add a dropout layer to randomly deactivate 50% of neurons during training
# This helps prevent overfitting and improves generalization
model.add(Dropout(0.5))

# Add the output dense layer with a softmax activation function
# The number of neurons is equal to the number of classes (derived from label_encoder.classes_)
# This produces probabilities for each class in a multi-class classification task
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

## Optimize model training
* for mixed precision (mixed_float16).

In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)

## Force stop
* check if the model is improving

In [17]:
early_stop = EarlyStopping(monitor='val_loss', # Monitor loss
                           patience=5, # Allow 5 epochs without improvements before stopping
                           restore_best_weights=True # Revert to best weights
            )

#Compiling and training

##Check GPU availability in TensorFlow

In [18]:
print("GPU available:", tf.config.list_physical_devices('GPU'))
print("Using GPU:", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU available: []
Using GPU: False


In [19]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.optimizer)

In [20]:
history = model.fit(train_dataset, epochs=10, validation_data=validation_dataset, callbacks=[early_stop])

Epoch 1/10
      2/Unknown 16s 6s/step - accuracy: 0.1250 - loss: 3.3688 

KeyboardInterrupt: 

##Check GPU usage after training

In [ ]:
!nvidia-smi

#Model evaluation

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
epochs = range(1, len(accuracy) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, accuracy, 'b', label='Accuracy on training data')
plt.plot(epochs, val_accuracy, 'r', label='Accuracy on validation data')
plt.title('Accuracy during training')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test_categorical)
print(f'Test accuracy: {accuracy:.4f}')

In [ ]:
y_pred_encoded = model.predict(X_test)
y_pred = np.argmax(y_pred_encoded, axis=1)
y_test = np.argmax(y_test_categorical, axis=1)

print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
conf_df = pd.DataFrame(conf_matrix, index=label_encoder.classes_, columns=label_encoder.classes_)

plt.figure(figsize=(10, 7))
sns.heatmap(conf_df, annot=True, cmap="Blues", fmt="d")
plt.title("Confuzion matrix")
plt.ylabel("Real classes")
plt.xlabel("Predicted classes")
plt.show()